# Multi sLM AI 업무비서
- 사용자 -> 오케스트레이터 모듈 -> 분기 (tool 집합) -> DB or WEB -> 응답 생성 -> 사용자

# Orchestraion 
- 키워드 추출
- 툴 분류

In [3]:
# !pip uninstall openai -y
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.14 requires openai<2.0.0,>=1.58.1, but you have openai 0.28.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from azure.gpt.orch import Orch

orch = Orch()

history = """
user : 한국 수도는 어디야?
"""
response = orch._generate(history)
print(response)

{"agent": "Web agent", "keyword": "한국 수도"}



In [2]:
history = """
user : 한국의 수도는 어디야?
assistant : 한국의 수도는 서울입니다.
user : 그럼 이 문서에서 그 정보 좀 찾아줄래? \n문서 이름 : "서울.pdf"
"""

response = orch._generate(history)
print(response)

{"agent": "Document: 서울.pdf", "keyword": "한국의 수도"}



# AI Search with MS Azure
- AI 기반 정보 검색 플랫폼
- 텍스트 검색, Vector 검색 및 하이브리드 검색 제공
    - BM25, HNSW, RRF 등 다양한 검색 알고리즘 제공
- 필터, 동의어 매핑 등 다양한 기능들을 쉽게 구현 가능
- 보안 및 암호화에 강점
    - 인바운드 및 아웃바운드 엑세스 보안

In [3]:
!pip install langchain==0.2.5
!pip install -U langchain-community==0.2.5
!pip install pypdf==4.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.6
    Uninstalling langsmith-0.2.6:
      Successfully uninstalled langsmith-0.2.6
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.28
    Uninstalling langchain-core-0.3.28:
      Successfully uninstalled langchain-core-0.3.28
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.4
    Uninstalling langchain-text-splitters-0.3.4:
      Successfully uninstalled langchain-text-splitters-0.3.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.13
    Uninstalling langchain-0.3.13:
      Successfully uninstalled langchain-0.3.13
ERROR: pip's depe

In [ ]:
#from google.colab import dirve
#drive.mount('/content/drive')

## PDF Splitter

In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

file_name = "기업공시서식 작성기준(2024.11.26. 시행).pdf"
file_path = os.getcwd() + f"/azure/cognitive_search/{file_name}"

loader = PyPDFLoader(file_path)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len
)

pages = loader.load_and_split(text_splitter)
len(pages)

1619

In [11]:
pages[100]

Document(metadata={'source': '/Users/wonik/Library/Mobile Documents/iCloud~md~obsidian/Documents/til/TIL/AI/slm_practice/project-multi_slm/azure/cognitive_search/기업공시서식 작성기준(2024.11.26. 시행).pdf', 'page': 19}, page_content='당 부분을 명시․참조하게 한다.\n ⅵ. 공모후 증권이 국내외 거래소에 상장되는지를 기재한다 . 이 경우 거래소 상장이 확정\n되었거나 , 상장될 것이라는 합리적 근거가 없는 한, 상장가능성에 대해 단정적으로 표\n현해서는 아니된다 .\nⅶ.일반청약자에 대한 환매청구권 부여여부는 한국거래소의 상장규정 또는 금융투자협회\n의 증권 인수업무 등에 관한 규정에 따라 일반청약자가 공모주식을 공모가격의 90% \n이상의 가격으로 인수인에게 매도할 수 있는 권리(이하 “환매청구권 ”이라 한다)의 부\n여여부 , 행사가능기간 등을 기재한다 .')

## Upload text data to AI Search

In [7]:
import re

def is_korean_english_ratio_high(text, threshold=0.8):
    # 한글과 영어 문자를 찾기 위한 정규식 표현
    korean_pattern = re.compile(r'[가-힣]')
    english_pattern = re.compile(r'[a-zA-Z]')

    # 한글과 영어 문자의 개수 세기
    korean_count = len(korean_pattern.findall(text))
    english_count = len(english_pattern.findall(text))

    # 전체 문자에서 한글과 영어 문자의 비율 계산
    total_count = len(text.replace(' ',''))
    if total_count == 0:
        return False # 텍스트가 비어있으면 False 반환
    
    ratio = (korean_count + english_count) / total_count
    return ratio >= threshold

    

In [3]:
from azure.cognitive_search.ai_search import AISearch

ai_search = AISearch()

In [ ]:
contexts = []

for page in pages:
    if is_korean_english_ratio_high(page.page_content):
        contexts.append({
            'context' : page.page_content,
            'file_name': file_name
        })

ai_search.upload(contexts)

## Keyword 검색

In [ ]:
search_documents = ai_search.search(
    query='noise',
    file_name=file_name,
    topk=3
)

print(search_documents)

# Bing Search API

In [1]:
!pip install openai==0.28


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Web API
from azure.bing_search.web import Web

web = Web()

web.search(query='한국')

[{'title': '대한민국 - 나무위키',
  'snippet': '한국 사회의 문제점 · 헬조선 · 국뽕 · 자국 혐오 ·니트족 · 열정페이 · 저출산 · 고령화 · 인종차별 · 난민 수용 논란 · 외국인 노동자 문제 · 지역 갈등 · 젠더 분쟁 · 성소수자 · 세대 갈등 · 강한 자만이 살아남는 90년대 · 인터넷 검열 · 권위주의· 황금 티켓 증후군',
  'url': 'https://namu.wiki/w/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD'},
 {'title': '한국 - 위키백과, 우리 모두의 백과사전',
  'snippet': '한국(韓國), 조선(朝鮮), 남북한(南北韓), 또는 코리아(영어: Korea)는 동아시아의 한반도(조선반도)에 위치한 지역 또는 나라를 지칭하는 말로, 오늘날에는 한반도와 그 부속 도서를 이르는 말이다.. 넓은 의미로 한국은 고조선 건국 이후 한반도에서 설립된 한민족 국가를 통칭하는 말이다.',
  'url': 'https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B5%AD'},
 {'title': '대한민국 - 위키백과, 우리 모두의 백과사전',
  'snippet': '대한민국(한국 한자: 大韓民國, 문화어: 남조선(2023년까지), 대한민국)은 동아시아의 한반도 군사 분계선 남부에 위치한 나라이다. 약칭으로 한국(한국 한자: 韓國)과 남한(한국 한자: 南韓)으로 부르며 현정체제는 대한민국 제6공화국이다. 대한민국의 국기는 대한민국 국기법에 따라 태극기  이며 ...',
  'url': 'https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD'}]

In [1]:
# News API
from azure.bing_search.news import News

news = News()
news.search("삼성전자")


[{'title': '"주가 하락하면 자사주도 덜 준다"…\'주가부양\' 칼빼든 삼성전자',
  'snippet': '삼성전자는 특히 약정 체결 이후 주가가 하락할 경우 하락률만큼 지급 주식 수량이 줄어들도록 했다. 삼성전자 관계자는 "지난 1년간의 성과에 따른 성과급을 받고 끝나는 게 아니라 향후 1년간 주가 하락에 대해서도 책임을 지도록 해 그만큼 임원의 책임 경영을 강화하고자 하는 것"이라고 설명했다.',
  'url': 'https://www.yna.co.kr/view/AKR20250117103800003'},
 {'title': '주가 떨어지면 삼성전자 임원 성과급도 감소.. 자사주 성과급 ...',
  'snippet': '주가 떨어지면 삼성전자 임원 성과급도 감소.. 자사주 성과급 제도 도입, 1년 뒤 지급 시점에 주가 10% 떨어지면 자사주 수량도 10% 줄어 책임 경영 ...',
  'url': 'https://www.hankyung.com/article/202501172747i'},
 {'title': '삼성전자, 임원 성과급 자사주로 지급한다…주가 하락 시 약정 ...',
  'snippet': '삼성전자 서울 서초 사옥. 박효상 기자 삼성전자가 임원의 성과급을 자사주로 지급한다. 책임경영 강화의 일환이다. 삼성전자는 17일 사내 게시판에 임원에 대한 초과이익성과급(opi)의 일부를 자사주로 지급하기로 했다는 내용을 공지했다.',
  'url': 'https://www.kukinews.com/article/view/kuk202501170130'}]